In [1]:
import data_cleaning as dc
import review_score_analysis as rs
import text_analysis as ml

cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.


In [2]:
# Load dataframes
chunk_size = 100_000
restaurants_df = dc.load("data/filtered_restaurants.json", chunk_size)
reviews_df = dc.load("data/filtered_reviews.json", chunk_size)
avg_scores_df = rs.calculate_average_review_score(reviews_df)

In [4]:
avg_scores_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4371248 entries, 0 to 4371247
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   review_id         object        
 1   user_id           object        
 2   business_id       object        
 3   stars             int64         
 4   useful            int64         
 5   funny             int64         
 6   cool              int64         
 7   text              object        
 8   date              datetime64[ns]
 9   avg_review_score  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 333.5+ MB


In [ ]:
# Train large 2 label classifier
X, y, classifier = ml.train_binary_model(avg_scores_df, 200_000)
ml.benchmark(X, y)
results = ml.evaluate_classifier(classifier, X, y)
print(f'Model Accuracy: {results}')

/home/chasty2/Documents/group-project-better-late-than-never/text_analysis.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(process)


Benchmark accuracy for our model to beat: 0.9477902610486948
Model Accuracy: 0.9974050129749351


In [4]:
# Cross validate kernel for binary classification
best_kernel = ml.binary_kernel_cross_validation(X, y)

print(f'the {best_kernel} kernel gives the highest accuracy')

SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
Accuracy of linear kernel on split 0: 0.9565808683826323
Accuracy of linear kernel on split 1: 0.95506
Accuracy of linear kernel on split 2: 0.95446
Accuracy of linear kernel on split 3: 0.9555
Accuracy of rbf kernel on split 0: 0.9569608607827843
Accuracy of rbf kernel on split 1: 0.95546
Accuracy of rbf kernel on split 2: 0.95476
Accuracy of rbf kernel on split 3: 0.95574
Accuracy of poly kernel on split 0: 0.9522609547809043
Accuracy of poly kernel on split 1: 0.95034
Accuracy of poly kernel on split 2: 0.94942
Accuracy of poly kernel on split 3: 0.95074
Accuracy of sigmoid kernel on split 0: 0.9558808823823524
Accuracy of sigmoid kernel on split 1: 0.95438
Accuracy of sigmoid kernel on split 2: 0.9536
Accuracy of sigmoid kernel on split 3: 0.95472
the rbf kernel gives the highest accuracy


In [5]:
# Save large 2-label classifier

## Note: cross-validation showed all kernels performing similarly, but 
## the poly kernel performed best with larger datasets
ml.save_model(X, y, classifier, "binary")

In [3]:
# Train and save large 3-label classifier
# GPU acceleration doesn't work on multiclas SVM, so we use less datapoints
X, y, classifier = ml.train_3_class_model(avg_scores_df, 100_000)

ml.save_model(X, y, classifier, "3_class")

/home/chasty2/Documents/group-project-better-late-than-never/text_analysis.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(process)


In [5]:
# Benchmark 3-way classifier
#X, y, classifier = ml.load_model("3_class")

ml.benchmark(X, y)
results = ml.evaluate_classifier(classifier, X, y)
print(f'Model Accuracy: {results}')



Benchmark accuracy for our model to beat: 0.5727342726572734
Model Accuracy: 0.997040029599704
